In [1]:

import langchain
import pinecone
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import Ollama
llm=Ollama(model="mistral")


d:\LC\myenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
import os


In [4]:
#Lets read a documnet
loader=PyPDFLoader("GIDC_case.pdf")
docs=loader.load()
docs

invalid pdf header: b'\xde"\x00\x00%'
incorrect startxref pointer(1)


[Document(metadata={'source': 'GIDC_case.pdf', 'page': 0}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 2 \nPETITIONER:\nGUJARAT INDUSTRIAL DEVELOPMENTCORPORATION\n        Vs.\nRESPONDENT:\nNAROTTAMBHAI MORARBHAI & ANR.\nDATE OF JUDGMENT:       23/08/1996\nBENCH:\nRAMASWAMY, K.\nBENCH:\nRAMASWAMY, K.\nG.B. PATTANAIK (J)\nACT:\nHEADNOTE:\nJUDGMENT:\n                         O R D E R\n     Delay condoned.\n     Leave granted.\n     Heard learned counsel on both sides.\n     Notification  under   Section  4   (1)  of   the   Land\nAcquisition Act,  1894  [for  short,  the  \'Act"]  was  last\npublished on April 10, 1989 acquiring a large extent of land\nadmeasuring  8,61,515  sq.mtr.  for  public  purpose,  viz.,\nindustrial development;  the land  was at  that time fit for\ncultivation  but   possessed  of  potential  value  and  was\nsituated in outskirts of Surat. The Land Acquisition Officer\nin his  award dated  April 15,  1991 awarded  compensation @\nRs.5/- p

In [5]:
#divide the docs into chunks
text_splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'GIDC_case.pdf', 'page': 0}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 2 \nPETITIONER:\nGUJARAT INDUSTRIAL DEVELOPMENTCORPORATION\n        Vs.\nRESPONDENT:\nNAROTTAMBHAI MORARBHAI & ANR.\nDATE OF JUDGMENT:       23/08/1996\nBENCH:\nRAMASWAMY, K.\nBENCH:\nRAMASWAMY, K.\nG.B. PATTANAIK (J)\nACT:\nHEADNOTE:\nJUDGMENT:'),
 Document(metadata={'source': 'GIDC_case.pdf', 'page': 0}, page_content='ACT:\nHEADNOTE:\nJUDGMENT:\n                         O R D E R\n     Delay condoned.\n     Leave granted.\n     Heard learned counsel on both sides.\n     Notification  under   Section  4   (1)  of   the   Land\nAcquisition Act,  1894  [for  short,  the  \'Act"]  was  last'),
 Document(metadata={'source': 'GIDC_case.pdf', 'page': 0}, page_content='published on April 10, 1989 acquiring a large extent of land\nadmeasuring  8,61,515  sq.mtr.  for  public  purpose,  viz.,\nindustrial development;  the land  was at  that time fit for\ncultivation  but

In [6]:
len(documents)

14

In [7]:
#Embedding Technique
embeddings=OllamaEmbeddings(model="mistral")
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='mistral', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [8]:
vector=embeddings.embed_query("The Land Acquisition Officer in his award dated April 15, 1991 awarded compensation @Rs.5/- per sq.mtr.")
print(vector)
len(vector)

[6.839293479919434, 1.5837205648422241, -3.2023613452911377, -0.4345974922180176, 10.056004524230957, 0.03904816508293152, 7.522355556488037, -1.2742040157318115, 1.0984737873077393, -2.0434422492980957, -5.30776309967041, -4.0970964431762695, 2.6575262546539307, -2.2256691455841064, 0.08166120201349258, -5.151590824127197, 0.08501231670379639, 0.11485125124454498, -1.4434415102005005, 3.0226755142211914, -3.622732400894165, -3.880444049835205, 5.285621643066406, 2.343756675720215, 0.3221636116504669, -6.041069984436035, 5.132381916046143, -7.7074384689331055, -6.998255729675293, -12.702826499938965, -0.6574803590774536, 1.2400202751159668, 0.9154594540596008, 0.8398239016532898, -12.018049240112305, -0.48924651741981506, 6.298598766326904, -2.3833491802215576, 8.243769645690918, 7.879525661468506, -2.711838722229004, -2.290104389190674, 3.878641366958618, 2.193509817123413, 1.9010872840881348, 4.052987575531006, 6.321115493774414, -5.788090705871582, -5.917370319366455, -5.63744735717

4096

In [12]:
#Vector search DB in python
from pinecone import Pinecone
# Pinecone.init( 
#         api_key="04dea1a1-a89b-4ecb-836c-2527158eae05",
#         region="us-east-1"
#        )
# index_name="ragtest"
pc = Pinecone(api_key="04dea1a1-a89b-4ecb-836c-2527158eae05")
index = pc.Index("ragtest")
